In [1]:
import numpy as np
from PIL import Image
import pandas as pd
%config Completer.use_jedi = False

In [2]:
image_names = !ls ../raw_data/galaxy-zoo-the-galaxy-challenge/images_test_rev1/ #loading images_names.jpg to list
df_image_names = pd.DataFrame(image_names)  
df_image_names = df_image_names.rename(columns={0:'image'}) 
df_training_solutions_rev1 = pd.read_csv('../raw_data/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.csv')
#loading y dataframe

In [3]:
df_training_solutions_rev1['image'] = df_image_names['image']
df_X_y = df_training_solutions_rev1.drop(columns=['GalaxyID'])
df_X_y.head(2) #final dataframe with y and image_names.jpg

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image
0,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.0,...,0.279952,0.138445,0.00000,0.0,0.092886,0.0,0.0,0.0,0.325512,100018.jpg
1,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.0,...,0.000000,0.131378,0.45995,0.0,0.591328,0.0,0.0,0.0,0.000000,100037.jpg


In [4]:
df_X_y_sample = df_X_y.sample(500)
df_X_y_sample.shape

(500, 38)

In [5]:
df_X_y_val = df_X_y.sample(500)

In [15]:
import tensorflow as tf

def load_image(image):
    '''returns one image'''
    folder = f'../raw_data/galaxy-zoo-the-galaxy-challenge/images_test_rev1/{image}'
    img = Image.open(folder)
    img_array = np.array(img)
    return np.resize(img_array, (224,224,3))
    
def load_images(df):
    '''returns array of images'''
    img_list = []
    for _, row in df.iterrows():
        img = load_image(row["image"])
        img_list.append(img) 
    return np.stack(img_list)

class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, df, batch_size=64, shuffle=True, ):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        '''returns number of minibatches per epoch'''
        return len(self.indices) // self.batch_size
    
    def on_epoch_end(self):
        '''shuffles the indices '''
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)
            
    def __get_X_image(self, df):
        '''returns images'''
        X = load_images(df)
        return X 
             
    def __get_Y(self, df):
        '''returns y'''
        return np.array(df.drop(columns=["image"]))
    
    def _get_data(self, batch):
        '''returns batch of images and y'''
        df_batch = self.df.query("index in @batch") 
        X = self.__get_X_image(df_batch)
        y = self.__get_Y(df_batch)
        return X, y
    
    def __getitem__(self, index):
        '''creates batches and returns final X and y'''
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        X, y = self._get_data(batch)
        return X, y

In [16]:
datagenerator = DataGenerator(df=df_X_y_sample, shuffle=False)

tr

In [17]:
datagenerator.__getitem__(0)[0].shape

AttributeError: 'DataGenerator' object has no attribute 'subset'

In [18]:
import keras 
import tensorflow.keras as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Concatenate
from keras import Sequential
from tensorflow.keras.models import Model

# telechargement modèle ResNet50
base_model = keras.applications.ResNet50(
        weights="imagenet",
        input_shape=(224, 224, 3),
        include_top=False
            )
# freeze 
for layer in base_model.layers[:150]:
    layer.trainable = False
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training =True)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(37, activation='linear')(x)
cnn_model = keras.Model(inputs=inputs, outputs=x)

In [19]:
cnn_model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 37)                75813     
Total params: 23,663,525
Trainable params: 10,065,957
Non-trainable params: 13,597,568
_________________________________________________________________


In [20]:
cnn_model.compile(loss='categorical_crossentropy',  #loss a voir
              optimizer='adam',
              metrics='accuracy')

In [21]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=20)

In [22]:
cnn_model.fit(datagenerator, batch_size=2, epochs=500, callbacks=es, verbose=1, validation_split=0.3)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class '__main__.DataGenerator'>]

In [23]:
datagenerator._get_data([1])

AttributeError: 'DataGenerator' object has no attribute 'subset'